In [ ]:
import cv2
import numpy as np

import email, smtplib, ssl
from getpass import getpass
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from os import listdir
from os.path import isfile, join

# Step-1: Cropping the face

In [ ]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions for  face cropping
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1 #count = count+1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# Step-2: Training our model for face Recognition using LBPH

In [ ]:
# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

# Step-3: Detecting the users face with the help of above created model

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = vimal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence >= 85:
            cv2.putText(image, "Hey Kartik", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
         
        else:
            
            cv2.putText(image, "I dont know,Who are u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 

# Step-4: Defining the Function to Send Mail and Whatsapp Message

In [ ]:
sender_email = "zxy@gmail.com"
receiver_email = "yzx@gmail.com"

def send_email():
    subject = "Security Alet!! Theft Detected..."
    body = """Hey,
    We found this person in front of your laptop.
    We informed you about this in case of security issues.
    The photo is attached below, recognize him."""
    password = getpass(prompt="Type your password and press enter:")

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    # message["Bcc"] = receiver_email  # Recommended for mass emails

    # Add body to email
    message.attach(MIMEText(body))
   # filename = "document.pdf"  # In same directory as script

    
    # Open PDF file in binary mode
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(open("detected.jpg", "rb").read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
    "Content-Disposition",
    f"attachment; filename= detected.jpg",)

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, text)
            server.quit()
            return 0
    except SMTPException as error:
        print ("Error!! Something went wrong")
        

# Creating A Fucntion for Sending Whatsapp Message!!
#pip install pywhatkit
#python -m pip install –-upgrade Pillow

import pywhatkit
import datetime
def send_webwhatsapp():
    pywhatkit.sendwhatmsg('+91xxxxxxxxxx', '''We found a person in front of your laptop. 
    We informed you about this in case of security issues Please Check your mail....The photo is attached below, recognize him.'''
                          ,datetime.datetime.now().hour, datetime.datetime.now().minute+2)
    return "Success"

# Step-5: Automatically Sending Mail and whatsapp Message if Face is Recoginized.

In [ ]:

# Open Webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = vimal_model.predict(face)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)      
        if confidence >=85:
            if cv2.imwrite("detected.jpg",image):
                print("One Face Successfuly detected")
                print("try sending the Image via Mail...")
                send=send_email()
                if send == 0:
                    print("Email Send Succesfully\t\t",end=" ")
                    whatsapp=send_webwhatsapp()
                    if whatsapp == "Success":
                        print("you will Recieve Whatsapp Message soon...")
                else:
                    print("Something went wrong while sending you a Email...!!")
            break
        
        else:
            print("Come infront of the camera.....")
            
    except:
        print("No face found..Try again later")
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

# Step-6: By Using BOTO3 Lib, Launching EC2, EBS and Attaching Both as Required.

In [ ]:
#EC2 Instance and Attaching a 5gb EBS Volume..
#################################################################################################
import boto3
import subprocess as sb
ec2 = boto3.resource('ec2')
# Connecting to AWS..We have to configure our AWS CLI first!!!

def EC2_launch():
    try:
        #LAunching an Ec2 Instance..
        instance = ec2.create_instances(
            ImageId='your image AMI id',
            MinCount=1,
            MaxCount=1,
            InstanceType='t2.micro',
            SecurityGroupIds=['users sg group(This is optional)'],
            SubnetId='users subnet id',)
        return (instance[0].id) 
    except:
        print("Configure your AWS CLI first!!\n")

###############################################################################################################################
# Creating a New EBS Volume..
def EBS_create():
    try:
        ec2 = boto3.resource('ec2')
        ebs = ec2.create_volume(AvailabilityZone='ap-south-1a', Size=5, VolumeType='gp2')
        return (ebs.id)
    except:
        print("Please check your internet Connectivity!!\n")
###############################################################################################################################
# Attaching NEW-EBS with the EC2 insatnce        
def Attach_EBS_to_EC2(ins_id,vol_id):
    volume = ec2.Volume(vol_id)
    attach_ebs= volume.attach_to_instance(
        Device='/dev/sdh',
        InstanceId=ins_id,
        VolumeId=vol_id,)
    return 0

#################################################################################################################################
import time
## Using Facial Recognition..For Sending Mail!!     
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

# Open Webcam
cap = cv2.VideoCapture(0)
f=0
trigger=0
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = vimal_model.predict(face)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)      
        if confidence >=85:
            cv2.imshow("face",face)
            f=f+1
        else:
            print("Come infront of the camera.....")
        if f==25:
            trigger=1
            print("One face detected...Launching EC2 instance")
            break
    except:
        pass
        
    if cv2.waitKey(10) == 13: #13 is the Enter Key
        break
               
cap.release()
cv2.destroyAllWindows()

if trigger==1:
    ec2_launch = EC2_launch()
    ebs=EBS_create()
    time.sleep(30)
    att=Attach_EBS_to_EC2(ec2_launch,ebs)